Gamma = Г(u) - число соседей вершины u (степень вершины u)
n = |V| - число вершин
m = |E_G| - число ребер
l=0.2
t_min - самая ранняя наблюдаемая отметка по всем ребрам сети
t_max - самая последняя наблюдаемая отметка по всем ребрам сети
t - временная метка ребра

Вектор признаков описывает ребро черех активности узлов, которое оно соединяет
Вектор признаков длиной 3*7*4 = 84

In [1]:
import numpy as np
import pandas as pd

In [169]:
def node_degree(numbers_of_nodes:np.array, adjacency_matrix:np.array):
    return np.sum(adjacency_matrix[numbers_of_nodes,:],axis=1)

def common_neighbours(u:int, v:int, adjacency_matrix:np.array) -> int:
    return np.sum(np.where(adjacency_matrix[u,:]+adjacency_matrix[v,:] != 0, 1, 0))

def adamic_adar(u:int, v:int, adjacency_matrix:np.array) -> float:
    return np.sum(1/node_degree(np.nonzero(adjacency_matrix[u,:]+adjacency_matrix[v,:]),adjacency_matrix))

def jaccard_coefficient(u:int, v:int, adjacency_matrix:np.array) -> float:
    return np.sum(np.where(adjacency_matrix[u,:]-adjacency_matrix[v,:] == 0, 1, 0))/np.sum(
        np.where(adjacency_matrix[u,:]+adjacency_matrix[v,:] != 0, 1, 0))

def preferential_attachment(u:int, v:int, adjacency_matrix:np.array) -> int:
    return np.prod(node_degree(np.array([u,v]),adjacency_matrix))

In [170]:
# 3 функции для вычисления весов

# Во все функции в качестве t можно передавать np массив временных меток всех ребер

def temporal_weighting_w_linear(t:np.array,t_min:int,t_max:int,l:float=0.2) -> float:
    return l+(1-l)*(t-t_min)/(t_max-t_min) # w_linear
def temporal_weighting_w_exponential(t:np.array,t_min:float,t_max:float,l:float=0.2) -> float:
    return l+(1-l)*(np.exp(3*(t-t_min)/(t_max-t_min))-1)/(np.exp(3)-1) # w_exponential 
def temporal_weighting_w_square_root(t:np.array,t_min:float,t_max:float,l:float=0.2) -> float:
    return l+(1-l)*np.sqrt((t-t_min)/(t_max-t_min)) #w_square_root

In [171]:
# 7 функций для аггрегации весов ребер, прилегающих к узлу

# edge_weights - np array размерности 2; i-ая строка - веса ребер, прилегающих к i-ой вершине 

def aggregation_of_node_activity_zeroth_quantile(edges_weights:list)-> float:
    return [np.quantile(weights_set,0) for weights_set in edges_weights]
def aggregation_of_node_activity_first_quantile(edges_weights:list)-> float:
    return [np.quantile(weights_set,0.25) for weights_set in edges_weights]
def aggregation_of_node_activity_second_quantile(edges_weights:list)-> float:
    return [np.quantile(weights_set,0.5) for weights_set in edges_weights]
def aggregation_of_node_activity_third_quantile(edges_weights:list)-> float:
    return [np.quantile(weights_set,0.75) for weights_set in edges_weights]
def aggregation_of_node_activity_fourth_quantile(edges_weights:list)-> float:
    return [np.quantile(weights_set,1) for weights_set in edges_weights]
def aggregation_of_node_activity_sum(edges_weights:list)-> float:
    return [np.sum(weights_set,0) for weights_set in edges_weights]
def aggregation_of_node_activity_mean(edges_weights:list)-> float:
    return [np.mean(weights_set,0) for weights_set in edges_weights]
    

In [172]:
# 4 функции для объединения статистик по парам узлов

def combining_node_activity_sum(aggregarion_node_one:np.array, aggregation_node_two:np.array)-> float:
    return aggregarion_node_one + aggregation_node_two
def combining_node_activity_absolute_diference(aggregarion_node_one:np.array, aggregation_node_two:np.array)-> float:
    return np.abs(aggregarion_node_one - aggregation_node_two)
def combining_node_activity_minimum(aggregarion_node_one:np.array, aggregation_node_two:np.array)-> float:
    return np.min(np.concatenate([aggregarion_node_one[:, np.newaxis], aggregation_node_two[:, np.newaxis]], axis=1), axis=1)
def combining_node_activity_maximum(aggregarion_node_one:np.array, aggregation_node_two:np.array)-> float:
    return np.max(np.concatenate([aggregarion_node_one[:, np.newaxis], aggregation_node_two[:, np.newaxis]], axis=1), axis=1)

In [173]:
def temporal_weighting(edge: pd.DataFrame, t_min: int, t_max: int):
    '''
    Взвешивание во времени: расчет трех весов для каждого ребра по их временным меткам
    '''
    edge['weight_linear'] = temporal_weighting_w_linear(edge['timestamp'],t_min,t_max)
    edge['weight_exponential'] = temporal_weighting_w_exponential(edge['timestamp'],t_min,t_max)
    edge['weight_square_root'] = temporal_weighting_w_square_root(edge['timestamp'],t_min,t_max)

In [174]:
def aggregation_of_node_activity(node: pd.DataFrame, edges_weights_for_node: pd.DataFrame):
    
    '''
    Агрегация активности узлов на основе 7 функций: 
    нулевой, первый,второй,третий,чертвертый квантили; сумма и среднее
    по весам ребер смежных с вершиной
    '''
    
    node['node_activity_zeroth_quantile_wl'] = aggregation_of_node_activity_zeroth_quantile(edges_weights_for_node["weight_linear"])
    node['node_activity_first_quantile_wl'] = aggregation_of_node_activity_first_quantile(edges_weights_for_node["weight_linear"])
    node['node_activity_second_quantile_wl'] = aggregation_of_node_activity_second_quantile(edges_weights_for_node["weight_linear"])
    node['node_activity_third_quantile_wl'] = aggregation_of_node_activity_third_quantile(edges_weights_for_node["weight_linear"])
    node['node_activity_fourth_quantile_wl'] = aggregation_of_node_activity_fourth_quantile(edges_weights_for_node["weight_linear"])
    node['node_activity_sum_wl'] = aggregation_of_node_activity_sum(edges_weights_for_node["weight_linear"])
    node['node_activity_mean_wl'] = aggregation_of_node_activity_mean(edges_weights_for_node["weight_linear"])
    
    node['node_activity_zeroth_quantile_we'] = aggregation_of_node_activity_zeroth_quantile(edges_weights_for_node["weight_exponential"])
    node['node_activity_first_quantile_we'] = aggregation_of_node_activity_first_quantile(edges_weights_for_node["weight_exponential"])
    node['node_activity_second_quantile_we'] = aggregation_of_node_activity_second_quantile(edges_weights_for_node["weight_exponential"])
    node['node_activity_third_quantile_we'] = aggregation_of_node_activity_third_quantile(edges_weights_for_node["weight_exponential"])
    node['node_activity_fourth_quantile_we'] = aggregation_of_node_activity_fourth_quantile(edges_weights_for_node["weight_exponential"])
    node['node_activity_sum_we'] = aggregation_of_node_activity_sum(edges_weights_for_node["weight_exponential"])
    node['node_activity_mean_we'] = aggregation_of_node_activity_mean(edges_weights_for_node["weight_exponential"])
    
    node['node_activity_zeroth_quantile_wsr'] = aggregation_of_node_activity_zeroth_quantile(edges_weights_for_node["weight_square_root"])
    node['node_activity_first_quantile_wsr'] = aggregation_of_node_activity_first_quantile(edges_weights_for_node["weight_square_root"])
    node['node_activity_second_quantile_wsr'] = aggregation_of_node_activity_second_quantile(edges_weights_for_node["weight_square_root"])
    node['node_activity_third_quantile_wsr'] = aggregation_of_node_activity_third_quantile(edges_weights_for_node["weight_square_root"])
    node['node_activity_fourth_quantile_wsr'] = aggregation_of_node_activity_fourth_quantile(edges_weights_for_node["weight_square_root"])
    node['node_activity_sum_wsr'] = aggregation_of_node_activity_sum(edges_weights_for_node["weight_square_root"])
    node['node_activity_mean_wsr'] = aggregation_of_node_activity_mean(edges_weights_for_node["weight_square_root"])
    

In [175]:
def combining_node_activity(node: pd.DataFrame)->pd.DataFrame:
    
    '''
    Объединение активности узлов для формирования векторного описания
    ребра на основе 4 функций:
    сумма, абсолютная разность, мин, макс 
    по парным активностям инцидентных вершин
    
    '''
    values = node[['node_activity_zeroth_quantile_wl',
    'node_activity_first_quantile_wl',
    'node_activity_second_quantile_wl',
    'node_activity_third_quantile_wl',
    'node_activity_fourth_quantile_wl',
    'node_activity_sum_wl',
    'node_activity_mean_wl',
    
    'node_activity_zeroth_quantile_we',
    'node_activity_first_quantile_we',
    'node_activity_second_quantile_we',
    'node_activity_third_quantile_we',
    'node_activity_fourth_quantile_we',
    'node_activity_sum_we',
    'node_activity_mean_we',
    
    'node_activity_zeroth_quantile_wsr',
    'node_activity_first_quantile_wsr',
    'node_activity_second_quantile_wsr',
    'node_activity_third_quantile_wsr',
    'node_activity_fourth_quantile_wsr',
    'node_activity_sum_wsr',
    'node_activity_mean_wsr']].values

    num_of_nodes = node.shape[0]
    num_of_feature = 21

    
    combined_values = []
    start_nodes_vector = []

    for i, row in enumerate(values):
        repeated_row = np.tile(row, num_of_nodes - i - 1)
        repeated_node = np.tile(i, num_of_nodes - i - 1)
        
        combined_values.extend(repeated_row)
        start_nodes_vector.extend(repeated_node)


    # Преобразование объединенных значений в массив NumPy
    first_combined_array = np.array(combined_values)
    
    end_nodes_vector = list(np.concatenate([np.arange(i, num_of_nodes) for i in range(1, num_of_nodes)]))
    
    # Замена номеров строк на значения из DataFrame и объединение в один вектор
    second_combined_array = np.concatenate([values[end_nodes_vector[i]] for i in range(len(end_nodes_vector))])
    
    feature_by_sym = combining_node_activity_sum(first_combined_array,second_combined_array)
    feature_by_abs_dif = combining_node_activity_absolute_diference(first_combined_array,second_combined_array)
    feature_by_min = combining_node_activity_minimum(first_combined_array,second_combined_array)
    feature_by_max = combining_node_activity_maximum(first_combined_array,second_combined_array)

    
    all_feature = [np.concatenate([feature_by_sym[i:i+num_of_feature], 
                                   feature_by_abs_dif[i:i+num_of_feature], 
                                   feature_by_min[i:i+num_of_feature], 
                                   feature_by_max[i:i+num_of_feature]]) 
                   for i in range(0, feature_by_sym.shape[0], num_of_feature)]
    
    feature_column_name = "feature_vector"
    Edge_feature = pd.DataFrame({"start_node":start_nodes_vector, "end_node":end_nodes_vector, feature_column_name: all_feature})
    return (Edge_feature,feature_column_name)

In [197]:
def combining_node_activity(node: pd.DataFrame)->pd.DataFrame:
    
    '''
    Объединение активности узлов для формирования векторного описания
    ребра на основе 4 функций:
    сумма, абсолютная разность, мин, макс 
    по парным активностям инцидентных вершин
    
    '''
    values = node[['node_activity_zeroth_quantile_wl',
    'node_activity_first_quantile_wl',
    'node_activity_second_quantile_wl',
    'node_activity_third_quantile_wl',
    'node_activity_fourth_quantile_wl',
    'node_activity_sum_wl',
    'node_activity_mean_wl',
    
    'node_activity_zeroth_quantile_we',
    'node_activity_first_quantile_we',
    'node_activity_second_quantile_we',
    'node_activity_third_quantile_we',
    'node_activity_fourth_quantile_we',
    'node_activity_sum_we',
    'node_activity_mean_we',
    
    'node_activity_zeroth_quantile_wsr',
    'node_activity_first_quantile_wsr',
    'node_activity_second_quantile_wsr',
    'node_activity_third_quantile_wsr',
    'node_activity_fourth_quantile_wsr',
    'node_activity_sum_wsr',
    'node_activity_mean_wsr']].values

    num_of_nodes = node.shape[0]
    num_of_feature = 21
    feature_column_name = "feature_vector"


    start_nodes_vector = np.repeat(np.arange(len(values)), num_of_nodes - np.arange(len(values)) - 1)
    
    end_nodes_vector = list(np.concatenate([np.arange(i, num_of_nodes) for i in range(1, num_of_nodes)]))
    
    first_combined_array = np.concatenate([values[start_nodes_vector[i]] for i in range(len(start_nodes_vector))])
    
    second_combined_array = np.concatenate([values[end_nodes_vector[i]] for i in range(len(end_nodes_vector))])
    
    feature_by_sym = combining_node_activity_sum(first_combined_array,second_combined_array)
    feature_by_abs_dif = combining_node_activity_absolute_diference(first_combined_array,second_combined_array)
    feature_by_min = combining_node_activity_minimum(first_combined_array,second_combined_array)
    feature_by_max = combining_node_activity_maximum(first_combined_array,second_combined_array)

    
    all_feature = [np.concatenate([feature_by_sym[i:i+num_of_feature], 
                                   feature_by_abs_dif[i:i+num_of_feature], 
                                   feature_by_min[i:i+num_of_feature], 
                                   feature_by_max[i:i+num_of_feature]]) 
                   for i in range(0, feature_by_sym.shape[0], num_of_feature)]
    
    Edge_feature = pd.DataFrame({"start_node":start_nodes_vector, "end_node":end_nodes_vector,feature_column_name:all_feature})

    return (Edge_feature,feature_column_name)

In [198]:
def combining_node_activity_for_absent_edge(node: pd.DataFrame, edge: pd.DataFrame)->pd.DataFrame:
    
    '''
    Объединение активности узлов для формирования векторного описания
    ребра на основе 4 функций:
    сумма, абсолютная разность, мин, макс 
    по парным активностям инцидентных вершин
    
    '''
    values = node[['node_activity_zeroth_quantile_wl',
    'node_activity_first_quantile_wl',
    'node_activity_second_quantile_wl',
    'node_activity_third_quantile_wl',
    'node_activity_fourth_quantile_wl',
    'node_activity_sum_wl',
    'node_activity_mean_wl',
    
    'node_activity_zeroth_quantile_we',
    'node_activity_first_quantile_we',
    'node_activity_second_quantile_we',
    'node_activity_third_quantile_we',
    'node_activity_fourth_quantile_we',
    'node_activity_sum_we',
    'node_activity_mean_we',
    
    'node_activity_zeroth_quantile_wsr',
    'node_activity_first_quantile_wsr',
    'node_activity_second_quantile_wsr',
    'node_activity_third_quantile_wsr',
    'node_activity_fourth_quantile_wsr',
    'node_activity_sum_wsr',
    'node_activity_mean_wsr']].values

    num_of_nodes = node.shape[0]
    num_of_feature = 21
    feature_column_name = "feature_vector"


    start_nodes_vector = np.repeat(np.arange(len(values)), num_of_nodes - np.arange(len(values)) - 1)
    
    end_nodes_vector = list(np.concatenate([np.arange(i, num_of_nodes) for i in range(1, num_of_nodes)]))
    
    Edge_feature = pd.DataFrame({"start_node":start_nodes_vector, "end_node":end_nodes_vector})

    # Оставляем только ребра, которых нет в статичном графе
    df_merged = Edge_feature.merge(edge[['start_node', 'end_node']], on=['start_node', 'end_node'], how='left', indicator=True)
    df_filtered = df_merged[df_merged['_merge'] == 'left_only']
    df_filtered = df_filtered.drop(columns='_merge')
    Edge_feature = df_filtered
    
    start_nodes_vector = Edge_feature['start_node'].values
    end_nodes_vector = Edge_feature['end_node'].values

    
    first_combined_array = np.concatenate([values[start_nodes_vector[i]] for i in range(len(start_nodes_vector))])
    
    second_combined_array = np.concatenate([values[end_nodes_vector[i]] for i in range(len(end_nodes_vector))])
    
    feature_by_sym = combining_node_activity_sum(first_combined_array,second_combined_array)
    feature_by_abs_dif = combining_node_activity_absolute_diference(first_combined_array,second_combined_array)
    feature_by_min = combining_node_activity_minimum(first_combined_array,second_combined_array)
    feature_by_max = combining_node_activity_maximum(first_combined_array,second_combined_array)

    
    all_feature = [np.concatenate([feature_by_sym[i:i+num_of_feature], 
                                   feature_by_abs_dif[i:i+num_of_feature], 
                                   feature_by_min[i:i+num_of_feature], 
                                   feature_by_max[i:i+num_of_feature]]) 
                   for i in range(0, feature_by_sym.shape[0], num_of_feature)]
    
    Edge_feature[feature_column_name] =  all_feature
    return (Edge_feature,feature_column_name)

In [199]:
def count_static_topological_features(df: pd.DataFrame):
    '''
    Рассчет статичных топологических признаков
    '''
    df["common_neighbours"] = df.apply(lambda row: common_neighbours(row["start_node"],row["end_node"],adjacency_matrix), axis=1)
    df["adamic_adar"] = df.apply(lambda row: adamic_adar(row["start_node"],row["end_node"],adjacency_matrix), axis=1)
    df["jaccard_coefficient"] = df.apply(lambda row: jaccard_coefficient(row["start_node"],row["end_node"],adjacency_matrix), axis=1)
    df["preferential_attachment"] = df.apply(lambda row: preferential_attachment(row["start_node"],row["end_node"],adjacency_matrix), axis=1)

In [200]:
def replace_nan(df, columns, start_node_column, end_node_column):
    '''
    Замена NaN на [] в числовых ячейках и сопоставление номеров для вершин,
    которые были только либо в end_node, либо в start_node
    '''
    
    for column in columns:
        df[column] = df[column].apply(lambda x: [] if not isinstance(x, list) else x)
    
    df[start_node_column] = np.where(np.isnan(df[start_node_column]), df[end_node_column], df[start_node_column])

    return df


In [201]:
def make_edges_weights_adjacent_to_node(edge: pd.DataFrame):
    
    '''
    Формирование датафрейма с весами примыкающих к вершине ребер.
    Строка соответствует определенной вершине.
    '''
    grouped_by_start_node = edge.drop(['end_node',"number","timestamp"], 
                                      axis=1).groupby("start_node").agg(
    {'weight_linear': list, 'weight_exponential': list,
     'weight_square_root': list}).reset_index()
    grouped_by_end_node = edge.drop(['start_node',"number","timestamp"], 
                                    axis=1).groupby("end_node").agg(
    {'weight_linear': list, 'weight_exponential': list,
     'weight_square_root': list}).reset_index()
    
    edges_weights_for_node = grouped_by_start_node.merge(
        grouped_by_end_node, left_on='start_node', right_on='end_node', how='outer')


    edges_weights_for_node = replace_nan(edges_weights_for_node,
                                         ['weight_linear_x', 'weight_exponential_x',
                                          "weight_square_root_x","weight_linear_y", 
                                          'weight_exponential_y', "weight_square_root_y"],
                                         'start_node',"end_node")
    
    edges_weights_for_node["weight_linear"]=edges_weights_for_node[
        "weight_linear_x"] + edges_weights_for_node["weight_linear_y"]
    edges_weights_for_node["weight_exponential"]=edges_weights_for_node[
        "weight_exponential_x"]+edges_weights_for_node["weight_exponential_y"]
    edges_weights_for_node["weight_square_root"]=edges_weights_for_node[
        "weight_square_root_x"]+edges_weights_for_node["weight_square_root_y"]
    edges_weights_for_node = edges_weights_for_node.drop(
        ["end_node",'weight_linear_x', 'weight_exponential_x',
         "weight_square_root_x","weight_linear_y", 'weight_exponential_y', 
         "weight_square_root_y"], axis=1)

    edges_weights_for_node['start_node'] = edges_weights_for_node['start_node'].astype(int)

    edges_weights_for_node[[
        "weight_linear","weight_exponential","weight_square_root"]] = edges_weights_for_node[[
        "weight_linear","weight_exponential","weight_square_root"]].apply(lambda x: np.array(x))

    edges_weights_for_node=edges_weights_for_node.sort_values(by='start_node')
    
    return edges_weights_for_node

In [202]:
def split_list_cell(df: pd.DataFrame, column_name:str):
    '''
    Разбиение списка на отдельные столбцы с автоматической генерацией имен
    '''
    return pd.concat([df.drop(column_name, axis=1),
                df[column_name].apply(lambda x: pd.Series(x))],
               axis=1)

In [203]:
def merge_with_temporal_graph_number(df: pd.DataFrame,node: pd.DataFrame):
    '''
    Сопоставление номера в статичном графе номеру из временного графа
    '''
    df = pd.merge(df, node[['number',"number_in_temporal_graph"]],
                            left_on='start_node', right_on='number')
    
    df = pd.merge(df, node[['number',"number_in_temporal_graph"]],
                            left_on='end_node', right_on='number')
    
    df = df.drop(["number_x","number_y"], axis=1)
    df = df.rename(columns={'number_in_temporal_graph_x': 'number_in_temporal_graph_start_node'})
    df = df.rename(columns={'number_in_temporal_graph_y': 'number_in_temporal_graph_end_node'})
    
    return df

In [204]:
def feature_for_edges(edge: pd.DataFrame, node: pd.DataFrame, adjacency_matrix: np.array, t_min:int, t_max:int):
    '''
    Получение датафрейма с признаками для ребер
    '''
    temporal_weighting(edge, t_min, t_max)
    
    edges_weights_adjacent_to_node = make_edges_weights_adjacent_to_node(edge)
    
    aggregation_of_node_activity(node,edges_weights_adjacent_to_node)
    
    Edge_feature,feature_column_name = combining_node_activity(node)

    Edge_feature = merge_with_temporal_graph_number(Edge_feature, node)
    
    count_static_topological_features(Edge_feature)
            
    Edge_feature = split_list_cell(Edge_feature, feature_column_name)

    return Edge_feature

In [205]:
def feature_for_absent_edges(edge: pd.DataFrame, node: pd.DataFrame, adjacency_matrix: np.array, t_min:int, t_max:int):
    '''
    Получение датафрейма с признаками для ребер
    '''
    temporal_weighting(edge, t_min, t_max)
    
    edges_weights_adjacent_to_node = make_edges_weights_adjacent_to_node(edge)
    
    aggregation_of_node_activity(node,edges_weights_adjacent_to_node)
    
    Edge_feature,feature_column_name = combining_node_activity_for_absent_edge(node,edge)

    Edge_feature = merge_with_temporal_graph_number(Edge_feature, node)
    
    count_static_topological_features(Edge_feature)
            
    Edge_feature = split_list_cell(Edge_feature, feature_column_name)

    return Edge_feature

In [213]:
Networks = ['email-Eu-core-temporal', 'munmun_digg_reply', 'opsahi-ucsocial','radoslaw_email','soc-sign-bitcoinalpha', 'sx-mathoverflow']

networks_files_names = [ f'datasets/{i}/out.{i}' for i in Networks]

In [221]:
number_of_datasets = 6
datasets_info = {'Network': ['email-Eu-core-temporal', 'munmun_digg_reply', 'opsahi-ucsocial','radoslaw_email','soc-sign-bitcoinalpha', 'sx-mathoverflow'],
'Label': ['EU','D-rep','UC','Rado','bitA','SX-MO'],
'Category': ['Social',"Social","Information","Social","Social","Social"],
'Edge type': ['Multi','Simple','Multi','Multi','Simple','Multi'],
'Path': networks_files_names}

datasets_info = pd.DataFrame(datasets_info)

In [222]:
datasets_info

,Network,Label,Category,Nodes,Edge type,Path
0,email-Eu-core-temporal,EU,Social,986,Multi,None
1,munmun_digg_reply,D-rep,Social,30398,Simple,None
2,opsahi-ucsocial,UC,Information,1899,Multi,None
3,radoslaw_email,Rado,Social,167,Multi,None
4,soc-sign-bitcoinalpha,bitA,Social,3783,Simple,None
5,sx-mathoverflow,SX-MO,Social,24818,Multi,None


In [227]:
# Таблица признаков для графа

def get_stats(network_file_name: str):
    
    tmpGraph = graphs.TemporalGraph(network_file_name)
    staticGraph = tmpGraph.get_static_graph(0.2, 0.6)
    snowball_sample_approach = graphs.SelectApproach(0, 5)
    random_selected_vertices_approach = graphs.SelectApproach()
    sg_sb = snowball_sample_approach(staticGraph.get_largest_connected_component())
    sg_rsv = random_selected_vertices_approach(staticGraph.get_largest_connected_component())
    
    # ск - снежный ком
    # свв - случайный выбор вершин
    return {
        'Сеть': network_file_name['Label'],
        'Категория': network_file_name['Category'],
        'Вершины': staticGraph.count_vertices(), 
        'Тип ребер': network_file_name['Edge type'],
        'Ребра':staticGraph.count_edges(),
        'Плотность графа':staticGraph.density(),
        'Доля вершин':staticGraph.share_of_vertices(),
        'Компоненты с/с':staticGraph.get_number_of_connected_components(),
        'Вершины в наибольшей компоненте с/с':staticGraph.get_largest_connected_component().count_vertices(),
        'Ребра в наибольшей компоненте с/с':staticGraph.get_largest_connected_component().count_edges(),
        'Радиус графа(ск)': staticGraph.get_radius(sg_sb),
        'Диаметр графа(ск)': staticGraph.get_diameter(sg_sb),
        '90 проц. расстояния(ск)': staticGraph.percentile_distance(sg_sb),
        'Радиус графа(свв)': staticGraph.get_radius(sg_rsv),
        'Диаметр графа(свв)': staticGraph.get_diameter(sg_rsv),
        '90 проц.расстояния(свв)': staticGraph.percentile_distance(sg_rsv),
        'Коэф.ассортативности': staticGraph.assortative_factor(),
        'Сред.класт.коэф.сети': staticGraph.average_cluster_factor(),
        'AUC': get_performance(tmpGraph),
    }

def graph_features_tables(datasets_info: pd.DataFrame):

    table = pd.DataFrame([get_stats(network) for index, network in datasets_info.iterrows()]).sort_values('Вершины')
    
    columns_to_include_to_feature_network_table = [
        'Сеть',
        'Категория',
        'Вершины', 
        'Тип ребер',
        'Ребра',
        'Плотность графа',
        'Доля вершин',
        'Компоненты с/с',
        'Вершины в наибольшей компоненте с/с',
        'Ребра в наибольшей компоненте с/с',
        'Радиус графа(ск)',
        'Диаметр графа(ск)',
        '90 проц. расстояния(ск)',
        'Радиус графа(свв)',
        'Диаметр графа(свв)',
        '90 проц.расстояния(свв)',
        'Коэф.ассортативности',
        'Сред.класт.коэф.сети',
    ]
    columns_to_include_to_auc_table = [
        'Сеть',
        'AUC',
    ]
    latex_feature_network_table = table.to_latex(
        formatters={
            'Вершины': lambda x: f'{x:,}', 
            'Ребра': lambda x: f'{x:,}',
            'Плотность графа': lambda x: f'{x:.2f}',
            'Доля вершин': lambda x: f'{x:.2f}',
            'Компоненты с/с': lambda x: f'{x:,}',
            'Вершины в наибольшей компоненте с/с': lambda x: f'{x:,}',
            'Ребра в наибольшей компоненте с/с': lambda x: f'{x:,}',
            'Радиус графа(ск)': lambda x: f'{x:.2f}',
            'Диаметр графа(ск)': lambda x: f'{x:.2f}',
            '90 проц. расстояния(ск)': lambda x: f'{x:.2f}',
            'Радиус графа(свв)': lambda x: f'{x:.2f}',
            'Диаметр графа(свв)': lambda x: f'{x:.2f}',
            '90 проц.расстояния(свв)': lambda x: f'{x:.2f}',
            'Коэф.ассортативности': lambda x: f'{x:.2f}',
            'Сред.класт.коэф.сети': lambda x: f'{x:.2f}',
        },
        column_format='l@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}r@{\hspace{1em}}r@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}c',
        index=False,
        caption=(
            "Признаки для сетей, рассмотренных в ходе работы "
        ),
        label='Таблица: Признаки сетей',
        escape=False,
        multicolumn=False,
        columns=columns_to_include_to_feature_network_table
    )
    latex_auc_table = table.to_latex(
        formatters={
            'AUC': lambda x: f'{x:.2f}',
        },
        column_format='l@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}r@{\hspace{1em}}r@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}c',
        index=False,
        caption=(
            "Точность пердсказания появления ребер"
        ),
        label='Таблица: AUC',
        escape=False,
        multicolumn=False,
        columns=columns_to_include_to_auc_table
    )
    return (latex_feature_network_table,latex_auc_table)